<a href="https://colab.research.google.com/github/Gabriel-Mourad/Madurity-lemon-estimator-TFG/blob/main/Mask_RCNN_TRAIN_Lemons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/TFG

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/TFG


In [ ]:
%tensorflow_version 1.x

In [ ]:
!pip list

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN
%cd Mask_RCNN
!pip3 install -3 requeriments.txt
!python3 setup.py install

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../

In [ ]:
import os 
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import imgaug



# Directorio de trabajo
ROOT_DIR = '/gdrive/MyDrive/TFG/projects/software/Mask_RCNN'


sys.path.append(ROOT_DIR)

# De la carpeta MRCNN importar:

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline

# Directorio para guardar el modelo con la configuración interna:
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Directrorio del modelo inicial pre-entrenado:
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "logs/mask_rcnn_coco.h5")


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [ ]:
# Definición de los directorios de datos:

DATA_DIR = "/gdrive/MyDrive/TFG/projects/data"

DATASET_TRAIN_DIR = os.path.join(DATA_DIR, 'train')
DATASET_VAL_DIR = os.path.join(DATA_DIR, 'val')
DATASET_TEST_DIR = os.path.join(DATA_DIR, 'test')

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)



/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


CONFIGURATION

In [ ]:
# Configuración para el entrenamieto con el conjunto de datos de limones:

class LemonsConfig( Config ):
  
  NAME = 'Lemons'
  IMAGES_PER_GPU = 1
  NUM_CLASSES = 1 + 1 + 1 + 1 # BACKGROUND + VERDE + MADURO + PODRIDO
  STEPS_PER_EPOCH = 1950 #numero de fotos que hay en el train set
  DETECTION_MIN_CONFIDENCE = 0.8

config = LemonsConfig()
config.display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.8
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
axis_Width = 15

DATASET


In [ ]:
class LemonsDataset(utils.Dataset):
    def load_dataset(self, dataset_dir):
        self.add_class('dataset', 1, 'VERDE')
        self.add_class('dataset', 2, 'MADURO')
        self.add_class('dataset', 3, 'PODRIDO')        
        # buscar todas las imagenes:
        for i, filename in enumerate(os.listdir(dataset_dir)):
            annotation_file = os.path.join(dataset_dir, filename.replace('.jpg', '.json'))
            if '.jpg' in filename and os.path.isfile(annotation_file):
                    self.add_image('dataset',
                               image_id=i,
                               path=os.path.join(dataset_dir, filename),
                               annotation=annotation_file)
   
# extracción de mascaras a cada imagen:
    def extract_masks(self, filename):
        json_file = os.path.join(filename)
        with open(json_file) as f:
            img_anns = json.load(f)
       
        n_masks = 0
        for anno in img_anns['shapes']:
            if anno['label']=='VERDE' or anno['label']=='MADURO' or anno['label']=='PODRIDO':
                n_masks+=1
           
        masks = np.zeros([img_anns['imageHeight'], img_anns['imageWidth'], n_masks], dtype='uint8')
        classes = []
        i=0
        for anno in img_anns['shapes']:
            if anno['label']=='VERDE' or anno['label']=='MADURO' or anno['label']=='PODRIDO':
                mask = np.zeros([img_anns['imageHeight'], img_anns['imageWidth']], dtype=np.uint8)            
                cv2.fillPoly(mask, np.array([anno['points']], dtype=np.int32), 1)
                masks[:, :, i] = mask
                classes.append(self.class_names.index(anno['label']))
                i+=1
        return masks, classes
   

    # lcargar las máscaras
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location
        path = info['annotation']
        # load masks
        masks, classes = self.extract_masks(path)
        return masks, np.asarray(classes, dtype='int32')
   
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']


1 executar codi # create
2 executar codi content/drive
3 tornar a executar codi #create

In [ ]:
# CREACION DEL DATASET

# train set

dataset_train = LemonsDataset()
dataset_train.load_dataset(DATASET_TRAIN_DIR)
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
  print("{:3}.  {:50}".format(i, info['name']))

# test set

dataset_test = LemonsDataset()
dataset_test.load_dataset(DATASET_TEST_DIR)
dataset_test.prepare()

print("Image Count: {}".format(len(dataset_test.image_ids)))
print("Class Count: {}".format(dataset_test.num_classes))
for i, info in enumerate(dataset_test.class_info):
  print("{:3}.  {:50}".format(i, info['name']))

# val set

dataset_val = LemonsDataset()
dataset_val.load_dataset(DATASET_VAL_DIR)
dataset_val.prepare()

print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
  print("{:3}.  {:50}".format(i, info['name']))


Image Count: 1950
Class Count: 4
  0.  BG                                                
  1.  VERDE                                             
  2.  MADURO                                            
  3.  PODRIDO                                           
Image Count: 557
Class Count: 4
  0.  BG                                                
  1.  VERDE                                             
  2.  MADURO                                            
  3.  PODRIDO                                           
Image Count: 278
Class Count: 4
  0.  BG                                                
  1.  VERDE                                             
  2.  MADURO                                            
  3.  PODRIDO                                           


In [ ]:
image_ids = np.random.choice(dataset_train.image_ids, 5)
for image_id in image_ids:
  image = dataset_train.load_image(image_id)
  mask, class_ids = dataset_train.load_mask(image_id)
  visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# Creación del modelo: seleccionar modo, configuración y directorio de trabajo
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)




In [ ]:
print(model.find_last())

/gdrive/MyDrive/TFG/projects/software/Mask_RCNN/logs/lemons20210608T0829/mask_rcnn_lemons_0009.h5


In [ ]:
%tensorboard --logdir /gdrive/MyDrive/TFG/projects/software/Mask_RCNN/logs

In [ ]:
# Cargar la configuración de pesos del modelo:

init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
   
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Cargar el ultimo modelo entrnado y continuar con el entrenamiento
    model.load_weights(model.find_last(), by_name=True)

Re-starting from epoch 100


TRAIN


In [ ]:
Lemon_augmentation = imgaug.augmenters.Sometimes(0.5,
    [imgaug.augmenters.geometric.Affine(rotate=(-360,360))])
    
    
model.train(dataset_train, dataset_test, 
            learning_rate=config.LEARNING_RATE, 
            epochs=100,
            layers='heads',augmentation = Lemon_augmentation)


Starting at epoch 9. LR=0.001

Checkpoint Path: /gdrive/MyDrive/TFG/projects/software/Mask_RCNN/logs/lemons20210608T0829/mask_rcnn_lemons_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (T

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/tensorflow-1.15.2/python3.7/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 10/100
1950/1950 [==============================] - 1114s 571ms/step - loss: 0.1160 - val_loss: 0.0692

Epoch 11/100
1950/1950 [==============================] - 1033s 530ms/step - loss: 0.1104 - val_loss: 0.0526
Epoch 12/100
1950/1950 [==============================] - 746s 383ms/step - loss: 0.0920 - val_loss: 0.0266
Epoch 13/100
1950/1950 [==============================] - 693s 356ms/step - loss: 0.0808 - val_loss: 0.0345
Epoch 14/100
1950/1950 [==============================] - 737s 378ms/step - loss: 0.0780 - val_loss: 0.0533
Epoch 15/100
1950/1950 [==============================] - 741s 380ms/step - loss: 0.0784 - val_loss: 0.0336
Epoch 16/100
1950/1950 [==============================] - 765s 392ms/step - loss: 0.0805 - val_loss: 0.0392
Epoch 17/100
1950/1950 [==============================] - 729s 374ms/step - loss: 0.0712 - val_loss: 0.0298
Epoch 18/100
1950/1950 [==============================] - 797s 409ms/step - loss: 0.0779 - val_loss: 0.0531
Epoch 19/100
1950/1950 [=

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=10, 
            layers="all")


Starting at epoch 100. LR=0.0001

Checkpoint Path: /gdrive/MyDrive/TFG/projects/software/Mask_RCNN/logs/lemons20210608T0829/mask_rcnn_lemons_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers ma